# Chapter3: Effective Functions

## 3.1 Python's Functions Are First-Class

“Python’s functions are first-class objects. You can assign them to variables, store them in data structures, pass them as arguments to other functions, and even return them as values from other functions.

Grokking these concepts intuitively will make understanding advanced features in Python like lambdas and decorators much easier. It also puts you on a path towards functional programming techniques.”

Excerpt From: Dan Bader. “Python Tricks: The Book.” Apple Books. 

In [1]:
def yell(text):
    return text.upper() + '!'

In [2]:
yell('hello')

'HELLO!'

### Functions Are Objects

“All data in a Python program is represented by objects or relations between objects. Things like strings, lists, modules, and functions are all objects. There’s nothing particularly special about functions in Python. They’re also just objects.

A variable pointing to a function and the function itself are really two separate concerns.”

Excerpt From: Dan Bader. “Python Tricks: The Book.” Apple Books. 

In [3]:
bark = yell

In [4]:
bark('woof')

'WOOF!'

In [5]:
del yell

In [6]:
yell('hello?')

NameError: name 'yell' is not defined

In [7]:
bark('hey')

'HEY!'

In [8]:
bark.__name__

'yell'

### Functions Can Be Stored in Data Structures

In [10]:
funcs = [bark, str.lower, str.capitalize]
funcs

[<function __main__.yell(text)>,
 <method 'lower' of 'str' objects>,
 <method 'capitalize' of 'str' objects>]

In [11]:
for f in funcs:
    print(f, f('hey there'))

<function yell at 0x7fcce09d9510> HEY THERE!
<method 'lower' of 'str' objects> hey there
<method 'capitalize' of 'str' objects> Hey there


In [12]:
funcs[0]('heyho')

'HEYHO!'

### Functions Can Be Passed to Other Functions

In [13]:
list(map(bark, ['hello', 'hey', 'hi']))

['HELLO!', 'HEY!', 'HI!']

### Functions Can Be Nested

In [14]:
def speak(text):
    def whisper(t):
        return t.lower() + '...'
    return whisper(text)

In [15]:
speak('Hello, World')

'hello, world...'

In [16]:
def get_speak_func(volume):
    def whisper(text):
        return text.lower() + '...'
    def yell(text):
        return text.upper() + '!'
    if volume > 0.5:
        return yell
    else:
        return whisper

In [17]:
get_speak_func(0.3)

<function __main__.get_speak_func.<locals>.whisper(text)>

In [18]:
get_speak_func(0.7)

<function __main__.get_speak_func.<locals>.yell(text)>

In [19]:
get_speak_func(0.7)('Hello')

'HELLO!'

### Functions Can Capture Local State

“A closure remembers the values from its enclosing lexical scope even when the program flow is no longer in that scope.”

Excerpt From: Dan Bader. “Python Tricks: The Book.” Apple Books. 

In [20]:
def make_adder(n):
    def add(x):
        return x + n
    return add

In [21]:
plus_3 = make_adder(3)

In [22]:
plus_3(4)

7

In [23]:
def make_adder2(n):
    a = 3
    def add(x):
        return x + a + n
    return add

In [27]:
plus_6 = make_adder2(3)

In [28]:
plus_6(4)

10

### Objects Can Behave Like Functions

“While all functions are objects in Python, the reverse isn’t true. Objects aren’t functions. But they can be made callable, which allows you to treat them like functions in many cases.

If an object is callable it means you can use the round parentheses function call syntax on it and even pass in function call arguments. This is all powered by the __call__ dunder method.”

Excerpt From: Dan Bader. “Python Tricks: The Book.” Apple Books. 

In [29]:
class Adder:
    def __init__(self, n):
        self.n = n
    
    def __call__(self, x):
        return self.n + x

In [31]:
plus_3 = Adder(3)
plus_3(4)

7

In [32]:
callable(plus_3)

True

In [34]:
callable('hello')

False

### Key Takeaways

- Everything in Python is an object, including functions. You can assign them to variables, store them in data structures, and pass or return them to and from other functions (first-class functions.)
- First-class functions allow you to abstract away and pass around behavior in your programs.
- Functions can be nested and they can capture and carry some of the parent function’s state with them. Functions that do this are called closures.
- Objects can be made callable. In many cases this allows you to treat them like functions.

## 3.2 Lambdas Are Single-Expression Functions

In [35]:
add = lambda x, y: x + y
add(5, 3)

8

In [36]:
(lambda x, y: x + y)(5, 3)

8

### But Maybe You Shouldn't...

In [37]:
#Harmful
list(filter(lambda x: x % 2 == 0, range(1, 6)))

[2, 4]

In [38]:
#Better
[x for x in range(16) if x % 2 == 0]

[0, 2, 4, 6, 8, 10, 12, 14]

### Key Takeaways

- Lambda functions are single-expression functions that are not necessarily bound to a name (anonymous).
- Lambda functions can’t use regular Python statements and always include an implicit return statement.
- Always ask yourself: Would using a regular (named) function or a list comprehension offer more clarity?

Excerpt From: Dan Bader. “Python Tricks: The Book.” Apple Books. 